<a href="https://colab.research.google.com/github/JonJonesBR/MESTRE_RPG_GEMINI/blob/main/MESTRE_RPG_GEMINI_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q google-generativeai

In [ ]:
import google.generativeai as genai
# import google.colab.userdata # Não será mais usado para obter a chave principal
import time
import os # Importado para interagir com o sistema de arquivos

API_KEY_FILENAME = ".gemini_api_key.txt" # Nome do arquivo para salvar a chave (ponto no início para ser "oculto")

DURATION_OPTIONS_CONFIG = {
    "1": {"name": "Curta (objetivo direto)", "id": "curta", "turns": 10},
    "2": {"name": "Média (mais exploração)", "id": "media", "turns": 20},
    "3": {"name": "Longa (desenvolvimento e clímax)", "id": "longa", "turns": 40}
}

def get_and_configure_api_key():
    """
    Obtém a chave API do Gemini, seja de um arquivo local ou pedindo ao usuário.
    Configura a API do Gemini.
    Retorna True se bem-sucedido, False ou SystemExit caso contrário.
    """
    api_key = None

    # 1. Tentar carregar a chave de um arquivo local
    if os.path.exists(API_KEY_FILENAME):
        try:
            with open(API_KEY_FILENAME, "r") as f:
                api_key = f.read().strip()
            if api_key and api_key.startswith("AIza") and len(api_key) > 20: # Validação básica
                print("Chave API carregada de arquivo local.")
            else:
                api_key = None # Chave inválida no arquivo
                if os.path.exists(API_KEY_FILENAME): # Se o arquivo existe mas a chave é ruim, remove o arquivo.
                    try:
                        os.remove(API_KEY_FILENAME)
                        print(f"Arquivo '{API_KEY_FILENAME}' continha uma chave inválida e foi removido.")
                    except Exception as e_rem:
                        print(f"Não foi possível remover o arquivo de chave inválida: {e_rem}")

        except Exception as e:
            print(f"Erro ao ler a chave API do arquivo '{API_KEY_FILENAME}': {e}")
            api_key = None

    # 2. Se não carregou do arquivo, pedir ao usuário
    if not api_key:
        print("\n--- Configuração da Chave API do Gemini ---")
        print("Para jogar, você precisará de uma chave API do Google Gemini.")
        print("Você pode obter uma gratuitamente em: https://aistudio.google.com/app/apikey")
        while True:
            entered_key = input("Por favor, insira sua chave API do Gemini: ").strip()
            if entered_key.startswith("AIza") and len(entered_key) > 20: # Validação básica
                api_key = entered_key
                # 3. Tentar salvar a chave para uso futuro
                try:
                    with open(API_KEY_FILENAME, "w") as f:
                        f.write(api_key)
                    print(f"Chave API válida recebida e salva localmente em '{API_KEY_FILENAME}'.")
                    print("Você não precisará inseri-la novamente neste ambiente se o arquivo persistir.")
                except Exception as e:
                    print(f"Não foi possível salvar a chave API no arquivo '{API_KEY_FILENAME}': {e}")
                    print("A chave será usada apenas para esta sessão, mas não foi salva para a próxima.")
                break
            else:
                print("Formato de chave API inválido. Uma chave válida geralmente começa com 'AIza' e é uma string longa.")
                retry = input("Deseja tentar inserir a chave novamente? (S/N): ").strip().upper()
                if retry != 'S':
                    print("Sem uma chave API válida, o jogo não pode continuar.")
                    return False # Usuário desistiu

    if not api_key:
        print("Chave API não configurada.")
        return False

    # 4. Configurar a API Gemini
    try:
        genai.configure(api_key=api_key)
        print("API do Gemini configurada com sucesso.")
        return True
    except Exception as e:
        print(f"Ocorreu um erro ao configurar a API do Gemini com a chave fornecida: {e}")
        print("Verifique se a chave API é válida e tem as permissões necessárias.")
        if os.path.exists(API_KEY_FILENAME): # Se a configuração falhar, a chave salva pode ser ruim
            try:
                os.remove(API_KEY_FILENAME)
                print(f"A chave API salva em '{API_KEY_FILENAME}' parece ser inválida e o arquivo foi removido.")
            except Exception as e_rem:
                print(f"Não foi possível remover o arquivo de chave inválida: {e_rem}")
        return False

# --- Início da Configuração do Script ---
if not get_and_configure_api_key():
    raise SystemExit("Falha na configuração da API Key. Encerrando o programa.")

generation_config = {
    "temperature": 0.8,
    "top_p": 0.95,
    "top_k": 40,
}

safety_settings = [
    {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
]

# CORREÇÃO IMPORTANTE: Usando um nome de modelo válido da API Gemini.
# "gemini-2.0-flash-lite" não é um modelo reconhecido.
model = genai.GenerativeModel(model_name="gemini-1.5-flash-latest",
                              generation_config=generation_config,
                              safety_settings=safety_settings)
print(f"Modelo Gemini ({model.model_name}) configurado e pronto para a aventura!")


# O restante do seu código (game_state, conversation_history, mestre_responde,
# iniciar_aventura, rodada_do_jogo, e o loop principal do jogo)
# permanece o mesmo que você forneceu na última mensagem.
# Cole ele aqui abaixo:

game_state = {
    "player_name": None,
    "player_class": None,
    "player_inventory": [],
    "current_location": None,
    "current_situation": None,
    "npcs_encountered": {},
    "plot_twists_triggered": [],
    "previous_player_actions": [],
    "world_lore": {
        "setting_description": None,
        "key_events_history": []
    },
    "current_objective": None,
    "chosen_duration_id": None,
    "chosen_duration_name": None,
    "max_turns": 0,
    "current_turn": 0
}

conversation_history = []

def mestre_responde(prompt_para_o_mestre):
    try:
        response = model.generate_content(prompt_para_o_mestre)
        return response.text
    except Exception as e:
        if "429" in str(e) or "quota" in str(e).lower():
            return f"Desculpe, aventureiro, parece que as energias místicas estão sobrecarregadas no momento (limite de cota da API atingido). Por favor, tente novamente mais tarde. (Erro: {e})"
        return f"Desculpe, aventureiro, parece que minhas energias místicas falharam por um momento. (Erro: {e})"

def iniciar_aventura():
    global game_state
    global conversation_history

    print("Bem-vindo, aventureiro! Antes de começarmos nossa jornada, conte-me sobre você.")

    player_name_input = input("Qual é o seu nome, ó valente? ")
    if not player_name_input.strip():
        game_state["player_name"] = "Aventureiro Anônimo"
        print("Vejo que prefere o mistério... 'Aventureiro Anônimo' será seu título por enquanto.")
    else:
        game_state["player_name"] = player_name_input

    player_class_options = ["Guerreiro Astuto", "Maga Perspicaz", "Ladina Silenciosa", "Explorador Corajoso", "Bardo Eloquente"]
    print("\nEscolha sua vocação entre os heróis de outrora:")
    for i, cls in enumerate(player_class_options):
        print(f"{i+1}. {cls}")

    class_choice_valid = False
    while not class_choice_valid:
        try:
            choice_input = input(f"Digite o número da sua escolha (1-{len(player_class_options)}): ")
            choice_index = int(choice_input) - 1
            if 0 <= choice_index < len(player_class_options):
                game_state["player_class"] = player_class_options[choice_index]
                class_choice_valid = True
            else:
                print(f"Número inválido. Por favor, escolha um número entre 1 e {len(player_class_options)}.")
        except ValueError:
            print("Entrada inválida. Por favor, digite apenas o número correspondente à sua classe.")

    print("\nEscolha a duração desejada para esta aventura:")
    for key, value in DURATION_OPTIONS_CONFIG.items():
        print(f"{key}. {value['name']} (~{value['turns']} turnos)")

    duration_choice_valid = False
    while not duration_choice_valid:
        choice_key = input(f"Digite o número da sua escolha (1-{len(DURATION_OPTIONS_CONFIG)}): ")
        if choice_key in DURATION_OPTIONS_CONFIG:
            chosen_option = DURATION_OPTIONS_CONFIG[choice_key]
            game_state["chosen_duration_id"] = chosen_option['id']
            game_state["chosen_duration_name"] = chosen_option['name']
            game_state["max_turns"] = chosen_option['turns']
            duration_choice_valid = True
        else:
            print("Escolha inválida. Tente novamente.")

    game_state["current_objective"] = "Entender onde você está e o que aconteceu para chegar aqui, e então buscar um desfecho para esta situação inicial."
    game_state["current_location"] = "Um local desconhecido e envolto em névoa."
    game_state["player_inventory"] = ["Um cantil com água", "Pederneira e isqueiro"]
    game_state["current_turn"] = 0

    print(f"\nSaudações, {game_state['player_name']}, o(a) {game_state['player_class']}!")
    print(f"Você escolheu uma aventura: {game_state['chosen_duration_name']}.")
    print("O ar crepita com magia ancestral... Prepare-se, pois sua aventura está prestes a começar!\n")

    prompt_inicial_contexto = f"""
    Você é um Mestre de RPG (Game Master - GM) experiente, narrando uma aventura de fantasia épica e imersiva.
    O jogador se chama {game_state['player_name']} e sua classe/vocação é {game_state['player_class']}.
    O jogador escolheu uma aventura com duração '{game_state['chosen_duration_name']}'. Isso significa que a aventura deve ter um arco narrativo com começo, meio e um clímax/resolução em cerca de {game_state['max_turns']} turnos de interação do jogador.
    O objetivo inicial do jogador é: {game_state['current_objective']}.
    O jogador começa com os seguintes itens: {', '.join(game_state['player_inventory'])}.

    Instruções para você, GM:
    1. Comece descrevendo vividamente o cenário inicial (2-3 parágrafos).
    2. Apresente uma situação intrigante ou um desafio imediato.
    3. Termine convidando o jogador à ação.
    4. Planeje uma narrativa que possa ser concluída dentro da duração escolhida.
    5. Sua primeira resposta deve ser apenas a narração inicial da aventura.

    GM, comece a aventura para {game_state['player_name']} agora, considerando a duração escolhida:
    """

    print("Mestre (Gemini): Tecendo os fios do destino e consultando os ventos da magia...")
    conversation_history = []
    descricao_inicial_aventura = mestre_responde(prompt_inicial_contexto)

    if "Erro:" in descricao_inicial_aventura or "429" in descricao_inicial_aventura or "quota" in descricao_inicial_aventura.lower():
        print("\n----------------------------------------------------------------------")
        print(f"Mestre (Gemini):\n{descricao_inicial_aventura}")
        print("----------------------------------------------------------------------")
        print("\nParece que os ventos da magia não estão favoráveis no momento.")
        # A mensagem de erro de mestre_responde já informa sobre cota ou erro genérico.
        # print("Por favor, verifique sua cota da API do Gemini e tente iniciar a aventura novamente mais tarde.")
        game_state["player_name"] = None
        return

    print("\n------------------------- SUA AVENTURA COMEÇA! -------------------------")
    print(descricao_inicial_aventura)
    print("----------------------------------------------------------------------")

    game_state["current_situation"] = descricao_inicial_aventura
    game_state["world_lore"]["setting_description"] = descricao_inicial_aventura
    # Adiciona o prompt inicial ao histórico para dar contexto ao Gemini sobre as instruções de setup
    conversation_history.append({'role': 'user', 'parts': [f"Instruções de Setup da Aventura: {prompt_inicial_contexto}"]})
    conversation_history.append({'role': 'model', 'parts': [descricao_inicial_aventura]})

def rodada_do_jogo():
    global game_state
    global conversation_history

    game_state["current_turn"] += 1
    print(f"\n--- Turno {game_state['current_turn']}/{game_state['max_turns']} ({game_state['chosen_duration_name']}) ---")

    acao_jogador = input(f"{game_state['player_name']}, o que você faz? (Digite 'sair' para terminar) ")

    if acao_jogador.lower() == 'sair':
        print(f"\nSua jornada termina aqui por enquanto, {game_state['player_name']}. Que seus feitos sejam lembrados!")
        return False

    game_state["previous_player_actions"].append(acao_jogador)
    if len(game_state["previous_player_actions"]) > 3:
        game_state["previous_player_actions"].pop(0)

    instrucao_duracao = ""
    if game_state["current_turn"] == game_state["max_turns"]:
        instrucao_duracao = f"ATENÇÃO MESTRE, ESTE É O ÚLTIMO TURNO ({game_state['current_turn']}/{game_state['max_turns']}) DA DURAÇÃO '{game_state['chosen_duration_name']}'! Sua resposta deve CONCLUIR o arco narrativo atual de forma satisfatória. Reaja à ação do jogador e, ao final da sua narração conclusiva, se possível e fizer sentido, deixe um gancho ou uma sugestão sutil de que a história poderia continuar ou se aprofundar, MAS sem fazer uma pergunta direta ao jogador sobre continuar ainda."
    elif game_state["current_turn"] == game_state["max_turns"] - 1 and game_state["max_turns"] > 1:
        instrucao_duracao = f"Atenção, Mestre: O próximo turno ({game_state['current_turn'] + 1}/{game_state['max_turns']}) será o último da duração '{game_state['chosen_duration_name']}'. Prepare a narrativa para um desfecho iminente na sua próxima resposta."
    elif game_state["current_turn"] == game_state["max_turns"] - 2 and game_state["max_turns"] > 2:
        instrucao_duracao = f"Atenção, Mestre: Estamos nos aproximando do fim da duração '{game_state['chosen_duration_name']}' (Turno {game_state['current_turn']} de {game_state['max_turns']}). Guie a narrativa para um clímax."
    elif game_state["current_turn"] > game_state["max_turns"] * 0.60:
        instrucao_duracao = f"Lembre-se, Mestre, esta é uma aventura de duração '{game_state['chosen_duration_name']}' (total de {game_state['max_turns']} turnos). Estamos no turno {game_state['current_turn']}. Progrida a narrativa para um clímax."

    historico_recente_formatado = ""
    if conversation_history:
        turns_to_include = 2
        start_index = max(0, len(conversation_history) - (turns_to_include * 2)) # Pega as últimas 'turns_to_include' interações (user+model)
        for i in range(start_index, len(conversation_history)):
            entry = conversation_history[i]
            part_content = str(entry['parts'][0] if isinstance(entry['parts'], list) and entry['parts'] else entry['parts'])
            # Evita incluir o prompt de setup gigante no histórico do prompt para o mestre
            if entry['role'] == 'user' and "Instruções de Setup da Aventura:" in part_content:
                continue # Pula o prompt de setup
            elif entry['role'] == 'user':
                historico_recente_formatado += f"Jogador fez: {part_content}\n"
            else: # model
                historico_recente_formatado += f"Mestre respondeu: {part_content}\n"


    prompt_para_mestre = f"""
    Você é um Mestre de RPG (GM) continuando uma aventura de fantasia.
    Jogador: {game_state['player_name']} ({game_state['player_class']}).
    Duração da Aventura Planejada: {game_state['chosen_duration_name']} (Total de turnos: {game_state['max_turns']}, Turno Atual: {game_state['current_turn']}).
    {instrucao_duracao}

    Histórico Recente da Conversa (últimas interações):
    {historico_recente_formatado if historico_recente_formatado else "Nenhuma interação anterior neste resumo."}

    Situação Atual Detalhada (última narração completa do Mestre): {game_state['current_situation']}
    Última ação específica do Jogador que você deve processar: '{acao_jogador}'
    Inventário do Jogador: {', '.join(game_state['player_inventory']) if game_state['player_inventory'] else 'Vazio'}
    Objetivo Atual do Jogador: {game_state['current_objective']}

    Instruções para você, Mestre:
    1. Reaja à "Última ação específica do Jogador" ('{acao_jogador}') de forma criativa e coerente com a "Situação Atual Detalhada" e o "Histórico Recente".
    2. Avance a narrativa.
    3. CONSIDERE A INSTRUÇÃO DE DURAÇÃO ACIMA. Se for o momento de concluir o arco atual, sua resposta DEVE levar a história a um final para esse arco.
    4. Termine sua narração normalmente (geralmente com uma deixa para o jogador), A MENOS que seja a conclusão absoluta da aventura e você esteja explicitamente encerrando tudo (conforme instrução de duração no turno {game_state['max_turns']}).

    Mestre, com base na ação '{acao_jogador}' e no contexto fornecido, como a história prossegue?
    """

    conversation_history.append({'role': 'user', 'parts': [acao_jogador]})
    print("\nMestre (Gemini): Analisando os caminhos do destino...")
    resposta_mestre = mestre_responde(prompt_para_mestre)

    if "Erro:" in resposta_mestre or "429" in resposta_mestre or "quota" in resposta_mestre.lower():
        print("\n----------------------------------------------------------------------")
        print(f"Mestre (Gemini):\n{resposta_mestre}")
        print("----------------------------------------------------------------------")
        print("\nO Mestre parece ter encontrado uma perturbação mágica. Tente sua ação novamente ou digite 'sair'.")
        if conversation_history and conversation_history[-1]['role'] == 'user':
            conversation_history.pop()
        game_state["current_turn"] -= 1
        return True

    print(f"\nMestre (Gemini):\n{resposta_mestre}\n")
    game_state["current_situation"] = resposta_mestre
    conversation_history.append({'role': 'model', 'parts': [resposta_mestre]})

    if game_state["current_turn"] >= game_state["max_turns"]:
        print(f"Você alcançou o final planejado para a aventura '{game_state['chosen_duration_name']}' (Turno {game_state['current_turn']}/{game_state['max_turns']}).")

        can_extend = False
        next_duration_key_str = None
        current_duration_key_str = None

        for key_str, val_dict in DURATION_OPTIONS_CONFIG.items():
            if val_dict["id"] == game_state["chosen_duration_id"]:
                current_duration_key_str = key_str
                break

        if current_duration_key_str == "1":
            can_extend = True
            next_duration_key_str = "2"
        elif current_duration_key_str == "2":
            can_extend = True
            next_duration_key_str = "3"

        if can_extend:
            while True:
                next_duration_details = DURATION_OPTIONS_CONFIG[next_duration_key_str]
                choice = input(f"O Mestre concluiu este arco. Você deseja estender sua aventura para '{next_duration_details['name']}' (~{next_duration_details['turns']} turnos totais)? (S/N): ").upper()
                if choice == "S":
                    game_state["chosen_duration_id"] = next_duration_details['id']
                    game_state["chosen_duration_name"] = next_duration_details['name']
                    game_state["max_turns"] = next_duration_details['turns']

                    game_state["current_objective"] = f"Continuar explorando após os eventos anteriores, agora em uma aventura '{game_state['chosen_duration_name']}'."

                    print(f"\nA aventura continua! Sua jornada foi estendida para: {game_state['chosen_duration_name']}.")
                    print(f"Você tem agora até o turno {game_state['max_turns']} para desvendar novos mistérios.")

                    return True
                elif choice == "N":
                    print("\nVocê escolheu encerrar a aventura. Que seus feitos sejam lembrados!")
                    return False
                else:
                    print("Escolha inválida. Digite S para Sim ou N para Não.")
        else:
            print("\nVocê completou a mais longa jornada planejada ou não há mais extensões disponíveis. A aventura se encerra aqui. Obrigado por jogar!")
            return False

    return True

# --- Bloco Principal de Execução da Aventura ---
# Coloque este bloco em uma célula separada no Colab para melhor controle.
# (O código do loop principal permanece o mesmo)
try:
     iniciar_aventura()
     if game_state["player_name"]: # Verifica se a aventura foi iniciada corretamente (sem erros de API na largada)
         jogando = True
         while jogando:
             jogando = rodada_do_jogo()
except NameError as e:
     print(f"Erro de Nome: Parece que alguma função essencial não foi definida. Verifique se a célula anterior com as definições foi executada. ({e})")
except SystemExit as e: # Captura os SystemExit que podem vir da configuração da API ou da desistência do usuário
     print(f"Execução interrompida: {e}")
except Exception as e:
     print(f"Ocorreu um erro inesperado e a aventura foi interrompida: {e}")
finally:
     print("\n--- Fim da Sessão de Jogo ---")